In [16]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd

In [5]:
import pandas as pd 
import itertools
import deans_factors
import warnings
import pickle
warnings.filterwarnings('ignore')

def calculate_probas(deans_factors,tmp_active,model):  
    probas = []
    for i, row in deans_factors.iterrows():
        probas.append(model.predict_proba([row[5:]])[0][1])
    return probas

def calculate_shapley(deans_factors,active_players):
    shapley_values={}
    for player in active_players:
        respective_player = deans_factors[(deans_factors['player1']==player) | (deans_factors['player2']==player) | (deans_factors['player3']==player) | (deans_factors['player4']==player) | (deans_factors['player5']==player) ]
        
        combinations = len(respective_player.index)
        number_players = len(active_players)
        
        shapley = 1/(combinations*number_players)*(respective_player['probas'].sum())
        
        shapley_values[player] = shapley
        
    return shapley_values

def to_shap(season,model):
    tmp_active = deans_factors.get_active_roster(season,roster)
    
    tmp_combinations = deans_factors.get_combinations(tmp_active)
    
    game_deans = pd.DataFrame(deans_factors.deans_factors_season(season,tmp_combinations))
    
    filled_game_deans = game_deans.fillna(0)
    
    return filled_game_deans
    # return filled_game_deans
    probas = calculate_probas(filled_game_deans,tmp_active,model)
    
    filled_game_deans['probas'] = probas
    
    shapley_values = calculate_shapley(filled_game_deans,tmp_active)
    
    return shapley_values

def calculate_probas_xgb(deans_factors,tmp_active,model):
    probas = []
    for i, row in deans_factors.iterrows():
        probas.append(model.predict_proba([row[5:]])[0][1])
    return probas

def to_shap_xgb(season,model):
    tmp_active = deans_factors.get_active_roster(season,roster)
    
    tmp_combinations = deans_factors.get_combinations(tmp_active)
    
    game_deans = pd.DataFrame(deans_factors.deans_factors_season(season,tmp_combinations))
    
    filled_game_deans = game_deans.fillna(0)
    
    filled_game_deans = filled_game_deans.rename(renaming,axis='columns')
    
    filled_game_deans_tmp = filled_game_deans.drop(['player1','player2','player3','player4','player5'], axis=1)
    
    probas = model.predict_proba(filled_game_deans_tmp)[:,1]
    
    filled_game_deans['probas'] = probas
    
    shapley_values = calculate_shapley(filled_game_deans,tmp_active)
    
    return shapley_values

# load season data for players 
season = pd.read_csv('lakers_season_18_19_absolute.csv')

# Get active roster for respective season 
roster = pd.read_csv('lakers_rosters.csv',sep=';')
roster = roster[roster['Season']=='18_19']['Player']

# Get active roster for the season --> Threshold of max 40 games missed!
active_roster = deans_factors.get_active_roster(season,roster)

# Get all possible lineup combinations with active roster
combinations = deans_factors.get_combinations(active_roster)

# calculate deans factors for each combination for the whole season 
deans_factors_season = deans_factors.deans_factors_season(season,combinations)

In [7]:
model_log_reg = pickle.load(open('logreg_game_outcome_v2.pkl', 'rb'))

In [14]:
len(combinations['LeBron James'])

210

In [10]:
season

,Unnamed: 0,Mike Muscala_twopointers,Mike Muscala_threepointers,Mike Muscala_misses_two,Mike Muscala_misses_three,Mike Muscala_opp_twopointers,Mike Muscala_opp_threepointers,Mike Muscala_opp_misses_two,Mike Muscala_opp_misses_three,Mike Muscala_turnovers,...,Michael Beasley_freethrows,Michael Beasley_misses_freethrow,Michael Beasley_offensive_rebounds,Michael Beasley_defensive_rebounds,Michael Beasley_opp_freethrows,Michael Beasley_opp_misses_freethrow,Michael Beasley_opp_offensive_rebounds,Michael Beasley_opp_defensive_rebounds,win,game_id
0,0,2.0,4.0,1.0,3.0,8.0,2.0,5.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21801219
1,1,2.0,4.0,2.0,2.0,15.0,10.0,5.0,17.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21801208
2,2,2.0,1.0,1.0,2.0,17.0,6.0,15.0,13.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21801192
3,3,1.0,3.0,2.0,2.0,19.0,13.0,18.0,26.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21801179
4,4,1.0,2.0,0.0,4.0,20.0,10.0,17.0,22.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21801161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21800082
75,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,15.0,3.0,7.0,23.0,1,21800066
76,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21800060
77,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,28.0,10.0,10.0,26.0,0,21800048


In [23]:
pd.read_csv('log_reg_shap_lakers_18_19.csv')

,Unnamed: 0,Lonzo Ball,Kentavious Caldwell-Pope,Tyson Chandler,Josh Hart,Brandon Ingram,LeBron James,Kyle Kuzma,JaVale McGee,Rajon Rondo,Lance Stephenson,Moritz Wagner
0,0,0.00055,0.000749,0.000997,0.000777,0.000863,0.001197,0.000767,0.001089,0.000405,0.000638,0.000751


In [24]:
pd.read_csv('xgbcl_shap_lakers_18_19.csv')

,Unnamed: 0,Lonzo Ball,Kentavious Caldwell-Pope,Tyson Chandler,Josh Hart,Brandon Ingram,LeBron James,Kyle Kuzma,JaVale McGee,Rajon Rondo,Lance Stephenson,Moritz Wagner
0,0,0.005561,0.009192,0.010325,0.008862,0.009621,0.011823,0.008688,0.010582,0.004481,0.007644,0.007983


In [25]:
pd.read_csv('dec_tree_shap_lakers_18_19.csv')

,Unnamed: 0,Lonzo Ball,Kentavious Caldwell-Pope,Tyson Chandler,Josh Hart,Brandon Ingram,LeBron James,Kyle Kuzma,JaVale McGee,Rajon Rondo,Lance Stephenson,Moritz Wagner
0,0,0.010999,0.014968,0.017846,0.015183,0.014687,0.022266,0.014461,0.023035,0.010562,0.014348,0.01496
